# Pose2Sim workshop
EnvisionBOX Summer School, 2025, Amsterdam. Presented by David Pagnon

<bt><br><br>

`Pose2Sim` provides a workflow for 3D markerless kinematics (human or animal), as an alternative to traditional marker-based MoCap methods. 

**Pose2Sim** is free and open-source, requiring low-cost hardware but with research-grade accuracy and production-grade robustness. It gives maximum control over clearly explained parameters. Any combination of phones, webcams, or GoPros can be used with fully clothed subjects, so it is particularly adapted to the sports field, the doctor's office, or for outdoor 3D animation capture.

***Note:*** For real-time analysis with a single camera, please consider **[Sports2D](https://github.com/davidpagnon/Sports2D)** (note that the motion must lie in the sagittal or frontal plane). 


In [ ]:
from IPython.display import Video, display

display(Video('Pose2Sim_small.mp4', embed=True, width=1280, height=720))

# Table of Contents

1. [Install and import libraries](#install-and-import-libraries)
2. [Install Pose2Sim](#install-pose2sim)
    1. [Install miniconda](#install-miniconda)
    2. [Create a new conda environment](#create-a-new-conda-environment)
    3. [Install OpenSim](#install-opensim)
    4. [Install Pose2Sim Core](#install-pose2sim-core)
    5. [Optional: Install GPU libraries](#optional-install-gpu-libraries)
3. [Try Pose2Sim Demo](#try-pose2sim-demo)
    1. [Copy demo files](#copy-demo-files)
    2. [Run Pose2Sim demo](#run-pose2sim-demo)
4. [Run on the Tragic Talkers dataset](#run-on-the-tragic-talkers-dataset)
    1. [Choose a subset](#choose-a-subset)
    2. [Download videos and calibration files](#download-videos-and-calibration-files)
    3. [Convert calibration files](#convert-calibration-files)
    4. [Copy configuration file](#copy-configuration-file)
    5. [Run pose estimation](#run-pose-estimation)
    6. [Run person association and triangulation](#run-person-association-and-triangulation)
    7. [Filter and run marker augmentation](#filter-and-run-marker-augmentation)
    8. [Run scaling and inverse kinematics](#run-scaling-and-inverse-kinematics)

## Install and import libraries

Install and import the libraries required for the notebook:

In [ ]:
!pip install ipywidgets ipyfilechooser opencv-python numpy
!jupyter nbextension enable --py widgetsnbextension

import os
from pathlib import Path
import shutil
import json
import cv2
import numpy as np

try: # For Google Colab
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError: # For Jupyter Notebook
    pass

## Install Pose2Sim

***N.B.*** *The following instructions are the same as those in the [Pose2Sim documentation](https://github.com/perfanalytics/pose2sim)*
<br>

1. Install [miniconda](https://docs.conda.io/en/latest/miniconda.html)
2. Create a new conda environment:

In [ ]:
!conda create -n Pose2Sim python=3.10 -y 
!conda activate Pose2Sim

3. Install OpenSim:

In [ ]:
!conda install -c opensim-org opensim -y

Also install the [OpenSim GUI](https://simtk.org/projects/opensim)

4. Install Pose2Sim:

In [ ]:
!pip install pose2sim

5. *Optional:* Install the libraries for GPU support:

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!pip uninstall onnxruntime
!pip install onnxruntime-gpu

## Try Pose2Sim Demo

Copy the Pose2Sim demo files to the current folder. Check how they are organized:

```
Trial
├── calibration \
├── videos \
└── Config.toml
```

In [ ]:
!conda activate Pose2Sim
from Pose2Sim import Pose2Sim

singleperson_demo_path = Path(Pose2Sim.__file__).parent.resolve() / 'Demo_SinglePerson'


!cp -r {singleperson_demo_path} {Path.cwd()}/
os.chdir(Path.cwd() / 'Demo_SinglePerson')

**Run Pose2Sim demo:** This runs all stages of the pipeline on the Demo data with the default parameters. We will elaborate about :
- **Calibration:** Determining the **intrinsic parameters** of the cameras (focal length, optical center, distortion, ...) and their **extrinsic parameters** (location and orientation).\ 
  Here, we just convert a calibration file. Proper calibration will be done this afternoon when we collect our own data
- **Pose estimation:** Detecting the points of interest in each video (joint centers, face, etc)
- **Synchronization:** Make sure that the same time frame corresponds to the same body position.\
  I did not manage to make the tool work in Jupyter, so we are just going to skip it for now. We will do it again from the command line when we collect our own data.
- **Person association:** Make sure that person 1 is associated with person 1 across all cameras
- **Triangulation:** Transform all the 2D positions on videos to a 3D position in meters
- **Filtering:** Results can be somewhat noisy, especially if we do not have many cameras, if they are not optimally placed, or if calibration is not perfect.
- **Marker augmentation:** Add markers to the skeleton to facilitate the inverse kinematics stage.
- **Kinematics:** Obtain a biomechanically consistent animated skeleton with adjusted and fixed limb lengths, and joint angle constraints.

In [ ]:
Pose2Sim.calibration()
Pose2Sim.poseEstimation()
# Pose2Sim.synchronization()
Pose2Sim.personAssociation()
Pose2Sim.triangulation()
Pose2Sim.filtering()
Pose2Sim.markerAugmentation()
Pose2Sim.kinematics()

# Or equivalently:
#Pose2Sim.runAll() 
# Or:
#Pose2Sim.runAll(do_calibration=True, do_poseEstimation=True, do_synchronization=True, do_personAssociation=True, do_triangulation=True, do_filtering=True, do_markerAugmentation=True, do_kinematics=True)

**Check results in OpenSim GUI:**

- File -> Open Model: Open the scaled `.osim` model from Demo_SinglePerson/kinematics
- File -> Load Motion: Load the `.mot` motion from Demo_SinglePerson/kinematics
- File -> Preview experimental data: Check the markers from Demo_SinglePerson/pose-3d

You can also open the .trc and .mot file in Excel or any other spreadsheet software.

## Run on the Tragic Talkers dataset


In [ ]:
os.chdir(Path.cwd().parent) # Go back to the original Day1_MotionTracking directory

### Choose a subset of the dataset to run on


In [ ]:
scene_name = 'femalemonologue2_t3' # Among 'conversation1_t3', 'femalemonologue1_t2', 'femalemonologue2_t3', 'interactive1_t2', 'interactive4_t3', 'male_monologue2_t3'*
cameras_to_use = ['01', '11', '12', '22'] # Among ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']

### Convert the calibration files to the Pose2Sim format

Create the Pose2Sim folder structure

In [ ]:
from Pose2Sim import Pose2Sim

sample_folder = Path.cwd().parent / 'Sample'
scene_folder = sample_folder/scene_name
calibration_folder = sample_folder/'camera_calibration_data'

Pose2Sim_scene_folder = Path.cwd() / ('Demo_'+scene_name)
Pose2Sim_scene_folder.mkdir(parents=True, exist_ok=True)
(Pose2Sim_scene_folder / 'calibration').mkdir(parents=True, exist_ok=True)
(Pose2Sim_scene_folder / 'videos').mkdir(parents=True, exist_ok=True)
demo_config_path = Path(Pose2Sim.__file__).parent.resolve() / 'Demo_SinglePerson' / 'Config.toml'
shutil.copyfile(demo_config_path, Pose2Sim_scene_folder / 'Config.toml')


In [ ]:
from Pose2Sim.calibration import toml_write
from Pose2Sim.common import rotate_cam, world_to_camera_persp

json_calib_files = sorted([list(calibration_folder.glob(f'*{nb}.json'))[0] for nb in cameras_to_use])
ret, C, S, D, K, R, T = [], [], [], [], [], [], []
for file_path in json_calib_files:
    with open(file_path, 'r') as f:
        calib_data_cam = json.load(f)['camera']
        C.append(file_path.stem)  # Name
        S.append([int(calib_data_cam['width']), int(calib_data_cam['height'])]) # Size
        D.append([float(d) for d in calib_data_cam['distortion']][:4]) # Distortion
        K.append(np.array([[float(calib_data_cam['fx']), float(calib_data_cam['skew']), float(calib_data_cam['cx'])],
                   [0, float(calib_data_cam['fy']), float(calib_data_cam['cy'])],
                   [0, 0, 1]])) # Intrinsics
        rot_mat_cam = np.array([float(r) for r in calib_data_cam['r']]).reshape(3,3)
        t_cam = np.array([float(t) for t in calib_data_cam['t']])

        # Rotate cameras by Pi/2 around x in world frame
        # camera frame to world frame
        R_w, T_w = world_to_camera_persp(rot_mat_cam, t_cam)
        # x_rotate -Pi/2 and z_rotate Pi
        R_w_90, T_w_90 = rotate_cam(R_w, T_w, ang_x=-np.pi/2, ang_y=0, ang_z=np.pi)
        # world frame to camera frame
        R_c_90, T_c_90 = world_to_camera_persp(R_w_90, T_w_90)
        # store in R and T
        R.append(cv2.Rodrigues(R_c_90)[0].squeeze())
        T.append(t_cam)

toml_calib_file = Pose2Sim_scene_folder/'calibration'/f"Calib_{'_'.join(cameras_to_use)}.toml"
toml_write(toml_calib_file, C, S, D, K, R, T)


### Run Pose2Sim with model with hands

**Open the Config.toml model, search for the section `pose` -> `pose_model`, and set it to "whole_body".**\
Save and exit


**Run Pose2Sim.**\
N.B.: Calibration does not need to be run as we just converted the calibration files.\
N.B.: Synchronization does not need to be run either as the cameras were already synchronized.

In [ ]:
os.chdir(Pose2Sim_scene_folder)
from Pose2Sim import Pose2Sim

# Pose2Sim.calibration()
Pose2Sim.poseEstimation()
# Pose2Sim.synchronization()
Pose2Sim.personAssociation()
Pose2Sim.triangulation()
Pose2Sim.filtering()
Pose2Sim.markerAugmentation()
Pose2Sim.kinematics()

**Check results in OpenSim GUI:**

- File -> Open Model: Open the scaled `.osim` model from Demo_SinglePerson/kinematics
- File -> Load Motion: Load the `.mot` motion from Demo_SinglePerson/kinematics
- File -> Preview experimental data: Check the markers from Demo_SinglePerson/pose-3d

You can also open the .trc and .mot file in Excel or any other spreadsheet software.

### Optional: Try it on a scene with multiple persons

Reproduce the previous steps on another scene\
Make sure you set `project` -> `multi_person` to "true" in COnfig.toml.


***What do you notice? Why did it not work well?***

<br>

**Answer:**\
We have plenty of cameras, but they are clustered in 2 almost identical locations.\
Use the [Pose2Sim Blender add-on](https://github.com/davidpagnon/Pose2Sim_Blender) to visualize it.

## Run on custom data

This time, let's follow the documentaiton on the Pose2Sim GitHub page: 

[Pose2Sim documentation](https://github.com/perfanalytics/pose2sim)